# 04 - Hypothesis Testing
---

### **Hypothesis Tests**
Hypothesis testing is a statistical method used to make decisions about a population based on sample data. The steps are as follows:
    
1. State the null hypothesis
   - This is a statement about the assumed current distribution of the data and acts as a default to be disproved
   
2. State the alternative hypothesis
   - This is a statement that there is an effect or a difference, it's what the test aims to support

3. Choose a significance level, $\alpha$
   - $\alpha$ is the probability of rejecting the null hypothesis when it is actually true. 
   - Commonly used values are 0.05 or 0.01

4. Collect data and calculate the test statistic
   - Data is collected from a sample, and a test statistic is calculated
   - The test statistic summarises the data in a way that allows for the testing of hypotheses

4. Determine the P-value
   - The P-value is the probability of obtaining a test statistic at least as extreme as the one observed, assuming the null 
   hypothesis is true.
   - Alternatively, a critical region can be determined, and the decision can be made based on whether the test statistic falls in the 
   critical region.

5. Reject or fail to reject the alternative hypothesis
   - If the P-value is less than the chosen significance level $\alpha$, reject the null hypothesis in favor of the alternative hypothesis
   - Otherwise, fail to reject the null hypothesis.

Hypothesis testing helps determine whether observed data deviates significantly from what is expected under the null hypothesis. In the event we reject the null hypothesis, hypothesis testing does not tell us what the new value of the parameter being tested should be, merely that is varies significantly from the null hypothesis.

The example below deomstrates a hypothesis test in which we test whether a coin flip is fair (P(heads) = P(tails) = 0.5). 

In [1]:
import scipy.stats as stats

# Define the null hypothesis proportion (fair coin)
p_null = 0.5

# Define the observed data
number_of_flips = 100
number_of_heads = 80  # Try different values

# Perform the binomial test by computing the p value which represents the chance, assuming the null hypothesis is true, that we get data
# at least as extreme as the observed data (i.e. more than number_of_heads heads)

# The alternative parameter specifies the alternative hypothesis:
# 'two-sided' means the alternative hypothesis is that the coin is not fair (p != 0.5)
binom_test = stats.binomtest(number_of_heads, number_of_flips, p_null, alternative='two-sided')

# Output the result
print(f"Number of flips: {number_of_flips}")
print(f"Number of heads: {number_of_heads}")
print(f"p-value: {binom_test.pvalue}")

# Determine the conclusion
alpha = 0.05  # significance level
if binom_test.pvalue < alpha:
    print("Reject the null hypothesis: The coin is not fair.")
else:
    print("Fail to reject the null hypothesis: The coin is fair.")

Number of flips: 100
Number of heads: 80
p-value: 1.1159089057251951e-09
Reject the null hypothesis: The coin is not fair.


### **Bootstrapping in Hypothesis Tests**
Suppose you have two datasets and want to assess whether their means are statistically different, but you're unsure about the underlying distributions — perhaps they appear non-normal or skewed. Traditional parametric tests like the t-test rely on assumptions about normality or equal variance, which may not hold true here. 

Bootstrapping is a non-parametric resampling technique that can help in this situation. It allows us to approximate the sampling distribution of a statistic (e.g., the mean) without assuming a specific form for the population distribution. We resample from our original dataset with replacement, compute sample means use these sample means to build a confidence interval which tells us whether the means or the original distributions are statistically different. 

In practice, it works as follows:
1. Sample from the two datasets with replacement to get n samples of size m for each dataset. Usually m is the size of the entire    dataset.  
2. Compute the means of each of the n samples
3. Subtract the means of the samples from one dataset from the means of the samples from the other
4. Empiraclly compute the 2.5th and 97.5th percentiles of the bootstrap distribution of mean differences to obtain a 95% confidence interval.
5. If this interval does not contain 0 then the means are statistically different at a 5% significance level. 

The example below compares the means accross two non-normal distributions using bootstrapping. 

In [1]:
import numpy as np

# 0: Generate Sample Data (can be non-normal)
A = np.random.beta(0.5, 0.5, 1000)
B = np.random.uniform(5, 10, 700)

# 1: Sample with replacement to get n bootstrap samples
n_samples = 10000
A_bootstrap = np.random.choice(A, (n_samples, 1000), replace=True)
B_bootstrap = np.random.choice(B, (n_samples, 700), replace=True)

# 2: Calculate the means of each bootstrap sample
A_sample_means = A_bootstrap.mean(axis=1)
B_sample_means = B_bootstrap.mean(axis=1)

# 3: Compute differences between sample means
differences = A_sample_means - B_sample_means

# 4: Empiraclly compute 95% confidence interval
confidence_interval_95 = np.percentile(differences, [2.5, 97.5])
print(f"95% Confidence Interval: {confidence_interval_95}")

if max(confidence_interval_95) < 0 or min(confidence_interval_95) > 0:
    print("The mean of dataset A is statistically different from the mean of dataset B at a 5% significance level")
else:
    print("The mean of dataset A is not statistically different from the mean of dataset B at a 5% significance level")

95% Confidence Interval: [-7.16947928 -6.95331734]
The mean of dataset A is statistically different from the mean of dataset B at a 5% significance level
